***

# **Analiza podatkov podcasta DTFH**

***


## Uvod

Duncan Trussell Family Hour (DTFH skrajšano) je podcast ameriškega avtorja Duncana Trussella, ki se ne preveč resno ukvarja s tematikami moderne duhovnosti. Gostje posameznih epizod so lahko vse od novinarjev, komediantov in glasbenikov do budističnih gurujev, psihoterapevtov ter izvajalcev poganskih spiritualnih obredov. Posamezna epizoda običajno poteka kot prost pogovor med Duncanom in njegovim gostom (ali gosti, v primeru ko se mu v eter pridruži več kot ena oseba). Pogoste tematike obsegajo življenske anekdote, mnenja o spiritualnosti, raznorazne pogovore o meditaciji ter uporabo halucinogenih psihoaktivnih substanc v namen spoznavanja sebe in sveta okoli nas.

V tem dokumentu bom analiziral nekatere podatke, ki sem jih zajel iz avtorjeve spletne strani ([DTFH](http://www.duncantrussell.com/episodes)). Začnimo najprej s potrebnim uvozom python paketa pandas, s katerim bom analizo izvajal, ter dokumenta DTFH.csv, v katerem so zbrani podatki posameznih epizod, potem pa si oglejmo kako izgledajo naši surovi podatki:

In [ ]:
# Importanje pandas in dsv datoteke
import pandas as pd
podcasti = pd.read_csv('C:/faks/programiranje 1/DTFH-analiza-podatkov/DTFH.csv')
pd.options.display.max_rows = 20

In [ ]:
# Ogled surovih podatkov
podcasti

## Trendi objavitev in trajanja epizod

### Povprečno trajanje epizode

Začnimo z izračunom povprečne dolžine epizode podcasta. Ker je v csv datoteki dolžina napisana v sekundah (saj je to format, v katerem je zapisana na itunesih) definirajmo še pomožno funkcijo, ki nam bo to dolžino zapisala v bolj predstavljivi obliki (ure, minute, sekunde).

In [ ]:
# Pomožna funkcija ter dodajanje novega stolpca v tabelo.
def sekunde_v_ure(n):
    h = (n // 3600)
    min = ((n // 60) - h * 60)
    sec = (n - 60 * min - 3600 * h)
    return (h, min, sec)

podcasti['dolzina_h'] = (podcasti.dolzina).apply(sekunde_v_ure)
podcasti

In [ ]:
# Izračun povprečja dolžine
pov_dolzina = round((podcasti.dolzina).mean())
print(sekunde_v_ure(pov_dolzina))

Povprečna dolžina epizode je torej 1h 34min.

### Razporeditev epizod glede na dolžino

Poglejmo si še koliko epizod je krajših od ene ure, koliko jih je med eno in dvema urama ter koliko jih je daljših od dveh ur.

In [ ]:
#Vse epizode razvrščene po dolžini
rezina_dolzin = (podcasti[['naslov', 'dolzina']]).copy()
rezina_dolzin['dolzina_h'] = (podcasti.dolzina).apply(sekunde_v_ure)
rezina_dolzin.sort_values('dolzina')

In [ ]:
#Epizode krajše od 1 ure
rezina_dolzin_kratke = rezina_dolzin[rezina_dolzin.dolzina < 3600]
rezina_dolzin_kratke.sort_values('dolzina')

In [ ]:
#Epizode med 1 in 2 urama
rezina_dolzin_pov = (rezina_dolzin[(rezina_dolzin.dolzina >= 3600)])[(rezina_dolzin.dolzina < 7200)]
rezina_dolzin_pov.sort_values('dolzina')

In [ ]:
#Epizode daljše od 2 ur
rezina_dolzin_dolge = rezina_dolzin[rezina_dolzin.dolzina >= 7200]
rezina_dolzin_dolge.sort_values('dolzina')

In [ ]:
#Preštete zgornje skupine epizod
st_kratkih = len(rezina_dolzin_kratke.index)
st_pov = len(rezina_dolzin_pov.index)
st_dolgih = len(rezina_dolzin_dolge.index)
print(st_kratkih, st_pov, st_dolgih)

Krajših od ene ure je torej le 7 epizod, daljših od dveh ur pa 57, s tem, da je najdaljša dolga več kot tri ure. Velika večina epizod (336) pa je dolgim nekaj med eno in dvema urama.



### Število epizod po mesecih in letih

Poglejmo si kdaj so bile epizode objavljene. V prvotni tabeli so časi objave napisani do dneva in celo ure natačno. Relevanten podatek bo kvečjemu katerega meseca in katerega leta je izšla epizoda, zatorej oblikujmo najprej funkciji, ki nam podata mesec in leto objave.

In [ ]:
#Definicija pomožne funkcije za mesece ter dodajanje stolpca 
def pretvori_v_mesece(str):
    return str[7:-8]

podcasti['mesec'] = (podcasti.datum).apply(pretvori_v_mesece)
podcasti

In [ ]:
#Definicija pomožne funkcije za leto ter dodajanje stolpca
def pretvori_v_leta(str):
    return str[4:]

podcasti['leto'] = (podcasti.mesec).apply(pretvori_v_leta)
podcasti


Oglejmo si sedaj graf, koliko epizod je bilo objavljenih v posameznem letu.

In [ ]:
#Graf epizod po letih
%matplotlib inline
podcasti_leta = podcasti.groupby('leto').size()
graf_epizode_po_letih = podcasti_leta.plot.bar()
graf_epizode_po_letih

Kot vidimo izide v povprečnem letu nekaj več kot 40 epizod. Podatki za leti 2013 in 2022 so nepopolni (večina epizod iz leta 2013 manjka, leto 2022 pa se je ravnokar začelo). Izjema je leto 2021 ko je izšlo skoraj 70 epizod. Morda je vzrok za to Covid 19. To hipotezo lahko podpremo/ovržemo z analizo izdanih epizod po mesecih, kjer se bo jasneje vidilo kdaj se je število izdanih epizod začelo večati.

In [ ]:
%matplotlib inline
podcasti_meseci = (podcasti[::-1]).groupby('mesec', sort= False).size()
graf_epizode_po_mesecih = podcasti_meseci.plot()
graf_epizode_po_mesecih

Kot vidimo noben mesec ne izstopa posebaj. Omembe vredno je da ni bilo meseca v zadnjih osmih letih, v katerem nista bili izdani vsaj dve epizodi podcasta. Število epizod se je začelo večati drugo polovico leta 2020 in potem ostalo visoko celo leto 2021.

### Spreminjanje trajanja epizod skozi leta

Zadnja stvari, ki si jo bomo ogledali je kako se je spreminjala povprečna dolžina epizod skozi leta obstoja podcasta. Po občutki bi predvideval, da se epizode v povprečju daljšajo.

## Analiza gostov

## Analiza opisov

## Eksplicitnost

Kot zadnjo zanimivost si oglejmo kako pogosto itunes označi epizode kot "eksplicitne".

In [ ]:
podcasti.groupby('eksplicitnost').size()

Kot kaže je le 7 epizod označenih kot eksplicitne. Ker jih je tako malo, lahko pogledamo katere natančno to so.

In [ ]:
rezina_eks = (podcasti[['naslov', 'eksplicitnost']]).copy()
eksplicitne_epizode = rezina_eks[rezina_eks.eksplicitnost == 'yes']
eksplicitne_epizode

## Zaključek